In [13]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import torch
import copy
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch import Tensor

In [14]:
train = pd.read_csv('dataAll.csv')
test = pd.read_csv('testAll.csv')
train_data=train.iloc[:,:30].applymap(lambda x: x+126.23).apply(lambda x: (x - x.mean()) 
                                                                / (x.std()))
train_loc = train.iloc[:,30:]

x_test = test.applymap(lambda x: x+126.23).apply(lambda x: (x - x.mean()) 
                                                                / (x.std()))
x_test = torch.from_numpy(
       x_test.values.astype(np.float32))

In [15]:
train_feat = torch.from_numpy(
        train_data.values.astype(np.float32))
train_labels = torch.from_numpy(
        train_loc.values.astype(np.float32))

In [16]:
num_train = int(0.8 * train_feat.shape[0]) # 划分训练样本和验证集样本
indices = np.arange( train_feat.shape[0])
np.random.shuffle(indices)  # shuffle 顺序
train_indices = indices[:num_train]
valid_indices = indices[num_train:]

train_features = train_feat[train_indices,:]
train_label = train_labels[train_indices,:]
valid_features  = train_feat[valid_indices,:]
valid_labels = train_labels[valid_indices,:]


In [17]:
feat_dim = train_data.shape[0]

In [18]:
def error(y_test,y_pre):
    y_test = np.array(y_test)
    y_pre = np.array(y_pre)
    m = y_test-y_pre
    m=np.square(m)
    m=m.sum(axis=1)
    m=np.sqrt(m)
    return m.sum()/len(y_test)

def get_model(feat_dim,output_dim = 2 ):
    net = nn.Sequential(
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim*4/5)),
        nn.BatchNorm1d(int(feat_dim*4/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*4/5),int(feat_dim*3/5)),
        nn.BatchNorm1d(int(feat_dim*3/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*3/5),int(feat_dim*2/5)),
        nn.BatchNorm1d(int(feat_dim*2/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*2/5),int(feat_dim/5)),
        nn.BatchNorm1d(int(feat_dim/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim/5), output_dim)
        )
    return net

def get_data(x,y,batch_size,shuffle):
    dataset = TensorDataset(x,y)
    return DataLoader(dataset, batch_size, shuffle=shuffle,num_workers=4)

def train_model(model,x_train,y_train,x_valid,y_valid,epochs,batch_size,lr,weight_decay,use_gpu):
    
    if use_gpu:
        model = model.cuda()
    train_data = get_data(x_train,y_train,batch_size,True)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss()
    #criterion = nn.CrossEntropyLoss()
    best_error = 0
    for e in  range(epochs):
        model.train()
        for data in train_data:
            x,y = data
            if use_gpu:
                x = x.cuda()
                y = y.cuda()
                
            out = model(x)
            loss = criterion(out, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if use_gpu:
                x_valid = x_valid.cuda()
        with torch.no_grad():
            output = model(x_valid)
        output = Tensor.cpu(output)
        out = output.numpy()
        print('epoch:{}'.format(e))
        error_i = error(out,y_valid)
        print('error:{}'.format(error_i)) 
        if e == 0:
            best_error = error_i
        elif error_i < best_error:
            best_model = copy.deepcopy(model)
    torch.save(best_model,"model_nn_std.t7")

def predict(x_test,use_gpu):
    net = torch.load('./model_nn_std.t7')
    if use_gpu:
        x_test = x_test.cuda()
    with torch.no_grad():
        a = Tensor.cpu(net(x_test)).numpy()
    #a = net(x_test).detach().numpy()
    #a = np.argmax(a,axis=1)
    np.savetxt('nn_std.txt',np.array(a),fmt='%f')
    #print(a)
    '''
    with open('./NN_master_ans.txt','w') as f:
        for i in a:
            f.write('{}'.format(i))
            f.write('\n')'''
    

In [19]:
# 可以调整的超参
batch_size = 128
epochs = 40000
lr = 0.001
wd = 0
use_gpu = True
feat_dim = train_feat.shape[1]
net = get_model(feat_dim)
train_model(net, train_features, train_label, valid_features, valid_labels, epochs, 
            batch_size, lr, wd, use_gpu)
predict(x_test,use_gpu)

epoch:0
error:37.30235789941785
epoch:1
error:37.227075416397156
epoch:2
error:37.163253961837
epoch:3
error:37.09983424967658
epoch:4
error:37.02479180142303
epoch:5
error:36.928123483990944
epoch:6
error:36.81884955126132
epoch:7
error:36.69673249514877
epoch:8
error:36.56540063065977
epoch:9
error:36.423032220245794
epoch:10
error:36.28428707147477
epoch:11
error:36.10313914941785
epoch:12
error:35.910151196636484
epoch:13
error:35.69429677393273
epoch:14
error:35.444407947930145
epoch:15
error:35.19974935316947
epoch:16
error:34.954984637774906
epoch:17
error:34.69772295439845
epoch:18
error:34.36739216122251
epoch:19
error:34.06216647800776
epoch:20
error:33.718391211190166
epoch:21
error:33.36624504770375
epoch:22
error:33.17843426584735
epoch:23
error:32.60167367399741
epoch:24
error:32.23730595084088
epoch:25
error:31.77857171733506
epoch:26
error:31.33959108182406
epoch:27
error:30.928201811125486
epoch:28
error:30.48559791397154
epoch:29
error:30.038799725097025
epoch:30
erro

epoch:242
error:2.609032950456824
epoch:243
error:2.240406189359638
epoch:244
error:2.21797730913446
epoch:245
error:2.1395806908453268
epoch:246
error:2.1398309902571153
epoch:247
error:2.2328150898488035
epoch:248
error:2.1724262903763747
epoch:249
error:2.085794742480595
epoch:250
error:2.1493913223641656
epoch:251
error:2.325227843527652
epoch:252
error:2.171430619845569
epoch:253
error:2.186398050917691
epoch:254
error:2.1037412892646343
epoch:255
error:2.1083485355352525
epoch:256
error:2.018721132660899
epoch:257
error:2.0195991545722833
epoch:258
error:2.033787420156856
epoch:259
error:2.131286384217335
epoch:260
error:2.477241861558053
epoch:261
error:2.2250575451770698
epoch:262
error:2.1707496791114167
epoch:263
error:2.061937813308538
epoch:264
error:2.0033615921632437
epoch:265
error:2.0640618051625164
epoch:266
error:2.022044982515362
epoch:267
error:2.043322169611093
epoch:268
error:1.926134985446313
epoch:269
error:1.9716812666760997
epoch:270
error:1.9406340328872898
e

epoch:478
error:1.631982011289214
epoch:479
error:1.5676133722105434
epoch:480
error:1.5698128486820828
epoch:481
error:1.45874055021022
epoch:482
error:1.5601876124373384
epoch:483
error:1.4836247334350743
epoch:484
error:1.5592838499555304
epoch:485
error:1.466760238114489
epoch:486
error:1.5760853361497413
epoch:487
error:1.6088570302393272
epoch:488
error:1.6523219573698253
epoch:489
error:1.5377865257115135
epoch:490
error:1.632190304616753
epoch:491
error:1.552470336756145
epoch:492
error:1.5417349397133733
epoch:493
error:1.5127894313955368
epoch:494
error:1.6691667129891656
epoch:495
error:1.7540174239974127
epoch:496
error:1.5746801852563066
epoch:497
error:1.5804992912657665
epoch:498
error:1.6141103174522962
epoch:499
error:1.5838364061994663
epoch:500
error:1.5431001374514877
epoch:501
error:1.6285717805021023
epoch:502
error:1.5054276282543662
epoch:503
error:1.4914118086998707
epoch:504
error:1.503735225228412
epoch:505
error:1.462476091273852
epoch:506
error:1.6005072945

epoch:714
error:1.5550636597469276
epoch:715
error:1.6280651808093467
epoch:716
error:1.4496348629022477
epoch:717
error:1.575912100531614
epoch:718
error:1.6220038291862064
epoch:719
error:1.6615099197525873
epoch:720
error:1.5419945562641495
epoch:721
error:1.4789399496179658
epoch:722
error:1.460501963231727
epoch:723
error:1.438411184609476
epoch:724
error:1.5258230034160738
epoch:725
error:1.577660406391494
epoch:726
error:1.4709864291922703
epoch:727
error:1.532994831672461
epoch:728
error:1.5199453099733182
epoch:729
error:1.5039090925169794
epoch:730
error:1.4967061544914295
epoch:731
error:1.5426571785555465
epoch:732
error:1.4561009573597186
epoch:733
error:1.5065779001253234
epoch:734
error:1.5089089219659606
epoch:735
error:1.536124916619502
epoch:736
error:1.4920960657038325
epoch:737
error:1.5058906426867724
epoch:738
error:1.4021610393252748
epoch:739
error:1.5553540702316462
epoch:740
error:1.4777045601552394
epoch:741
error:1.472285775287031
epoch:742
error:1.503165616

epoch:951
error:1.39599640958522
epoch:952
error:1.374035597155967
epoch:953
error:1.425915795803687
epoch:954
error:1.4614925803990135
epoch:955
error:1.4087735234071799
epoch:956
error:1.4135849571474774
epoch:957
error:1.38106293079924
epoch:958
error:1.4776944534282017
epoch:959
error:1.3958839722469276
epoch:960
error:1.3939971726431113
epoch:961
error:1.4776413931112549
epoch:962
error:1.4844068993572122
epoch:963
error:1.456241661950194
epoch:964
error:1.361523311165912
epoch:965
error:1.4057118167852523
epoch:966
error:1.3735564751273448
epoch:967
error:1.419140814501132
epoch:968
error:1.46529097267141
epoch:969
error:1.924007519404916
epoch:970
error:1.6499909355291882
epoch:971
error:1.5757925569008733
epoch:972
error:1.4793099505781049
epoch:973
error:1.5409025559912677
epoch:974
error:1.5396749044914295
epoch:975
error:1.436071003547461
epoch:976
error:1.4646861482252587
epoch:977
error:1.5573402000121281
epoch:978
error:1.4519838873504205
epoch:979
error:1.528519288688551

...

epoch:38956
error:1.0904589970084089
epoch:38957
error:1.087060452124434
epoch:38958
error:1.1006089934710543
epoch:38959
error:1.086483105342416
epoch:38960
error:1.0880920489115056
epoch:38961
error:1.1001736146203913
epoch:38962
error:1.1158410944574708
epoch:38963
error:1.1030982487568726
epoch:38964
error:1.0845528783958602
epoch:38965
error:1.085338676423027
epoch:38966
error:1.0719381828357455
epoch:38967
error:1.0990343969821312
epoch:38968
error:1.0934314012219033
epoch:38969
error:1.1146474742025791
epoch:38970
error:1.143741740908999
epoch:38971
error:1.071626690350097
epoch:38972
error:1.0912984870229625
epoch:38973
error:1.079727547905886
epoch:38974
error:1.0935216511359962
epoch:38975
error:1.1350757751859637
epoch:38976
error:1.1356143531947365
epoch:38977
error:1.0996341680647639
epoch:38978
error:1.086202485749515
epoch:38979
error:1.0924941602067837
epoch:38980
error:1.1130384516746847
epoch:38981
error:1.0937686342779753
epoch:38982
error:1.071179230802272
epoch:389

epoch:39180
error:1.0788405246907342
epoch:39181
error:1.198714361153784
epoch:39182
error:1.1458530203953752
epoch:39183
error:1.127668728649539
epoch:39184
error:1.1451750800958118
epoch:39185
error:1.1142709196616267
epoch:39186
error:1.1315005204964423
epoch:39187
error:1.1553392891433538
epoch:39188
error:1.0892063945261967
epoch:39189
error:1.104105447273205
epoch:39190
error:1.1091517834583198
epoch:39191
error:1.1350037647558215
epoch:39192
error:1.1043360069837485
epoch:39193
error:1.093257612892141
epoch:39194
error:1.0987971257731646
epoch:39195
error:1.08333264902369
epoch:39196
error:1.1024953982808456
epoch:39197
error:1.1178621240297542
epoch:39198
error:1.09765688167044
epoch:39199
error:1.2028743847529917
epoch:39200
error:1.1131148048391009
epoch:39201
error:1.123360104579358
epoch:39202
error:1.1004584979887613
epoch:39203
error:1.1079452929182163
epoch:39204
error:1.0897279963918984
epoch:39205
error:1.1168292639018031
epoch:39206
error:1.0891587034079884
epoch:3920

epoch:39403
error:1.1186371836594438
epoch:39404
error:1.1458683384035413
epoch:39405
error:1.1407928664193887
epoch:39406
error:1.1054205851289618
epoch:39407
error:1.164758206030684
epoch:39408
error:1.1105270089646668
epoch:39409
error:1.089916865853412
epoch:39410
error:1.1177398957996443
epoch:39411
error:1.1220492304990701
epoch:39412
error:1.1409197532189925
epoch:39413
error:1.1226595031027653
epoch:39414
error:1.1310885924108587
epoch:39415
error:1.1139745083077297
epoch:39416
error:1.1093821062924483
epoch:39417
error:1.0779483691532583
epoch:39418
error:1.096102024882762
epoch:39419
error:1.0987790442068242
epoch:39420
error:1.0890510825567998
epoch:39421
error:1.130525300296127
epoch:39422
error:1.1001525326194614
epoch:39423
error:1.0910432921652653
epoch:39424
error:1.0914396653662677
epoch:39425
error:1.123787350673108
epoch:39426
error:1.108491924725097
epoch:39427
error:1.1448706149438066
epoch:39428
error:1.2058112575042448
epoch:39429
error:1.133052771643556
epoch:39

epoch:39626
error:1.1136626210280562
epoch:39627
error:1.1381100041437582
epoch:39628
error:1.160266081697728
epoch:39629
error:1.1211783938389392
epoch:39630
error:1.1234380369198738
epoch:39631
error:1.120068233040912
epoch:39632
error:1.108186354149822
epoch:39633
error:1.1264792142525064
epoch:39634
error:1.1199248438510672
epoch:39635
error:1.1309686539860933
epoch:39636
error:1.184241054283231
epoch:39637
error:1.140767915437015
epoch:39638
error:1.1213401014715394
epoch:39639
error:1.1416413577377102
epoch:39640
error:1.1324205484920764
epoch:39641
error:1.1005832529006307
epoch:39642
error:1.1086969807416316
epoch:39643
error:1.1042921058881792
epoch:39644
error:1.1361519836978493
epoch:39645
error:1.1227496740580531
epoch:39646
error:1.1150931176978898
epoch:39647
error:1.1377216057820585
epoch:39648
error:1.1092969097418741
epoch:39649
error:1.1279066315289457
epoch:39650
error:1.1071793135510997
epoch:39651
error:1.0959745064127182
epoch:39652
error:1.1179822203721297
epoch:

epoch:39849
error:1.1207289603209896
epoch:39850
error:1.103369077457956
epoch:39851
error:1.104862346377749
epoch:39852
error:1.1082805520041639
epoch:39853
error:1.0922984213292368
epoch:39854
error:1.0958523571414134
epoch:39855
error:1.1152217415912031
epoch:39856
error:1.1083040817280483
epoch:39857
error:1.1425195375667043
epoch:39858
error:1.1288152894566623
epoch:39859
error:1.0961888795682406
epoch:39860
error:1.097920604079075
epoch:39861
error:1.1139858783756469
epoch:39862
error:1.12003649160131
epoch:39863
error:1.1424385258327943
epoch:39864
error:1.1354850976309832
epoch:39865
error:1.1414729386066866
epoch:39866
error:1.1006742924027733
epoch:39867
error:1.1435944827377102
epoch:39868
error:1.6370352168903621
epoch:39869
error:1.406498246482859
epoch:39870
error:1.358833500515443
epoch:39871
error:1.2946649430485933
epoch:39872
error:1.2744742291093951
epoch:39873
error:1.2449021037152328
epoch:39874
error:1.255810420540912
epoch:39875
error:1.2024318206510753
epoch:398

In [20]:
y_valid = valid_labels#.cuda()
x_valid = valid_features.cuda()
net = torch.load('model_nn_std.t7')
with torch.no_grad():
    output_1 = net(x_valid)
output_1 = Tensor.cpu(output_1)  
out_1 = output_1.numpy()
error(out_1,y_valid)

1.0875356262128073